# Web Scraping

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import numpy as np

In [ ]:
url = "https://results.eci.gov.in/PcResultGenJune2024/index.htm"
r = requests.get(url)
html_content = r.content
# print(html_content)

In [ ]:
soup = BeautifulSoup(html_content,'html.parser')
# print(soup.prettify)

In [ ]:
title = soup.title
print(title)
# print(type(soup))
# print(type(title.string))
# type(title)

In [ ]:
#print(soup.find('table'))

In [ ]:
data = []
for row in soup.find_all('tr'):
    cols = row.find_all(['td', 'th'])
    if len(cols) == 4 and cols[0].text.strip() != 'Total':
        party = cols[0].text.strip()
        won = cols[1].text.strip()
        leading = cols[2].text.strip()
        total = cols[3].text.strip()
        data.append([party, won, leading, total])

print(f"Extracted data: {data}")

In [ ]:
with open('election_results.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    writer.writerows(data)

In [ ]:
try:
    df = pd.read_csv('election_results.csv', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv('election_results.csv', encoding='ISO-8859-1')


In [ ]:
print(f"DataFrame shape: {df.shape}") 
print(f"DataFrame contents:\n{df}")

# Top Insights


In [ ]:
insights = []

# 1. Total number of parties
insights.append(f"1. Total number of parties: {len(df)}")

# 2. Party with the most seats
most_seats_party = df.loc[df['Total'].idxmax(), 'Party']
most_seats = df['Total'].max()
insights.append(f"2. Party with the most seats: {most_seats_party} ({most_seats} seats)")

# 3. Party with the least seats
least_seats_party = df.loc[df['Total'].idxmin(), 'Party']
least_seats = df['Total'].min()
insights.append(f"3. Party with the least seats: {least_seats_party} ({least_seats} seats)")

# 4. Total seats won
total_seats_won = df['Won'].sum()
insights.append(f"4. Total seats won: {total_seats_won}")

# 5. Percentage of seats won by the leading party
leading_party_percentage = (most_seats / total_seats_won) * 100
insights.append(f"5. Percentage of seats won by the leading party: {leading_party_percentage:.2f}%")

# 6. Average seats won per party
avg_seats = df['Won'].mean()
insights.append(f"6. Average seats won per party: {avg_seats:.2f}")

# 7. Difference in seats between top two parties
df_sorted = df.sort_values('Total', ascending=False)
seat_difference = df_sorted.iloc[0]['Total'] - df_sorted.iloc[1]['Total']
insights.append(f"7. Difference in seats between top two parties: {seat_difference}")

# 8. Parties with more than 50 seats
parties_over_50 = df[df['Total'] > 50]['Party'].tolist()
insights.append(f"8. Parties with more than 50 seats: {', '.join(parties_over_50)}")

top_5_parties = df_sorted.head(5)[['Party', 'Total']].values.tolist()
insights.append("9. Top 5 parties by total seats:")
for i, (party, seats) in enumerate(top_5_parties, start=1):
    insights.append(f"    {i}. {party}: {seats} seats")
    
# 10. Percentage of total seats for each party
df['Percentage'] = (df['Total'] / df['Total'].sum()) * 100
percentages = df[['Party', 'Percentage']].values.tolist()
insights.append("10. Percentage of total seats for each party:")
for party, percentage in percentages:
    insights.append(f"   - {party}: {percentage:.2f}%")

# Print insights
print("\nInsights:")
for insight in insights:
    print(insight)

# Visualizations


In [ ]:
plt.style.use('seaborn-darkgrid')
colors = plt.cm.get_cmap('tab20')(np.linspace(0, 1, len(df)))
custom_palette = sns.color_palette(colors)

# Total seats won by each party

In [ ]:
plt.rcParams['figure.figsize'] = [30, 20]

# 1. Bar chart of total seats won by each party
plt.figure()
ax = sns.barplot(x='Party', y='Total', data=df, palette=custom_palette)
plt.title('Total Seats Won by Each Party', fontsize=20)
plt.xlabel('Party', fontsize=16)
plt.ylabel('Total Seats', fontsize=16)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
for i, v in enumerate(df['Total']):
    ax.text(i, v, str(int(v)), ha='center', va='bottom', fontsize=12)
plt.tight_layout()
plt.show()

# Top 6 parties

In [ ]:
df_sorted = df.sort_values('Total', ascending=False).reset_index(drop=True)

top_6 = df_sorted.head(6)
others = pd.DataFrame({
    'Party': ['Others'],
    'Total': [df_sorted.iloc[6:]['Total'].sum()]
})

pie_data = pd.concat([top_6, others]).reset_index(drop=True)

plt.style.use('seaborn-darkgrid')

colors = plt.cm.get_cmap('tab10')(np.linspace(0, 1, len(pie_data)))
custom_palette = sns.color_palette(colors)

plt.figure(figsize=[12, 8])

wedges, texts, autotexts = plt.pie(pie_data['Total'], 
                                   labels=None,  
                                   autopct=lambda pct: f'{pct:.1f}%\n' if pct > 2 else '',
                                   startangle=90, 
                                   colors=custom_palette,
                                   wedgeprops=dict(width=0.6)) 

plt.title('Seat Distribution Among Top 6 Parties and Others', fontsize=20)

plt.setp(autotexts, size=10, weight="bold",color="white")
plt.setp(texts, size=12)

centre_circle = plt.Circle((0,0), 0.3, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.axis('equal')  
plt.tight_layout()

plt.legend(wedges, pie_data['Party'],
           title="Parties",
           loc="center left",
           bbox_to_anchor=(1, 0, 0.5, 1))

plt.show()